<a href="https://colab.research.google.com/github/ke-wen/ke-wen/blob/main/Milestone/Whole_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Reading the dataset

# New Section

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks")
!pwd

/content/drive/MyDrive/Colab Notebooks


In [ ]:
pip install pandas

In [ ]:
import pandas as pd

In [ ]:
# Step 1: Load the Dataset
dataset_path = 'data/HateSpeechDatasetBalanced.csv'
df = pd.read_csv(dataset_path)
print('The shape of the dataset is:', df.shape)

The shape of the dataset is: (726119, 2)


In [ ]:
df.head(10)

,Content,Label
0,denial of normal the con be asked to comment o...,1
1,just by being able to tweet this insufferable ...,1
2,that is retarded you too cute to be single tha...,1
3,thought of a real badass mongol style declarat...,1
4,afro american basho,1
5,yeah retard haha,1
6,the ching chong chung stuff,1
7,the dead what a slut still warm when she tweet...,1
8,let your tweets be harmless it will not affect...,1
9,these latinos who have a problem with immigrat...,1


2. Exploratory data analysis

In [ ]:
df["Label"].value_counts()

,count
Label,
1,364525
0,361594


In [ ]:
print(df.isna().sum())

Content    0
Label      0
dtype: int64


3. Text Processing: Stop words removal and stemming

In [ ]:
pip install transformers torch scikit-learn

4. Train - Validation Split

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Split the dataset into train and test sets
train_data, test_data, train_labels, test_labels = train_test_split(df["Content"], df['Label'], test_size=0.1, random_state=42)

5.Use BERT for text embeddings

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
from tqdm import tqdm


# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# BERT-based Classifier
# Load BERT tokenizer and model, move to GPU
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained("bert-base-uncased").to(device)

# Tokenize and encode the text data, move to GPU
max_length = 128
X_train_tokens = tokenizer(list(train_data), truncation=True, padding=True, max_length=max_length, return_tensors="pt", add_special_tokens=True).to(device)
X_test_tokens = tokenizer(list(test_data), truncation=True, padding=True, max_length=max_length, return_tensors="pt", add_special_tokens=True).to(device)

# Calculate BERT embeddings for the text data
def get_bert_embeddings(tokens):
    embeddings = []
    for i in tqdm(range(len(tokens['input_ids']))):
        with torch.no_grad():
            output = bert_model(input_ids=tokens['input_ids'][i].unsqueeze(0), attention_mask=tokens['attention_mask'][i].unsqueeze(0))
        embeddings.append(output[0].squeeze().mean(dim=0).cpu().numpy())
    return embeddings


X_train_bert_embeddings = get_bert_embeddings(X_train_tokens)
X_test_bert_embeddings = get_bert_embeddings(X_test_tokens)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

100%|██████████| 72612/72612 [12:37<00:00, 95.87it/s]


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Initialize and train the KNN classifier
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train_bert_embeddings, train_labels)

NB = GaussianNB()
NB.fit(X_train_bert_embeddings, train_labels)

# Train a classifier on BERT embeddings (you can use any classifier of your choice)
# Here, we'll use Logistic Regression as an example
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_bert_embeddings, train_labels)

#rf=RandomForestClassifier()
#rf.fit(train_embeddings, train_labels)
#xgb=GradientBoostingClassifier()
#xgb.fit(train_embeddings, train_labels)

LogisticRegression(max_iter=1000)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Make predictions on the test data
predictions = knn.predict(X_test_bert_embeddings)

# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions)
print(f'KNN Accuracy: {accuracy * 100:.2f}%')
#predictions = rf.predict(test_embeddings)
predictions = NB.predict(X_test_bert_embeddings)
accuracy = accuracy_score(test_labels, predictions)
print(f'Naive Bais Accuracy: {accuracy * 100:.2f}%')

#predictions = rf.predict(test_embeddings)
predictions = lr.predict(X_test_bert_embeddings)
accuracy = accuracy_score(test_labels, predictions)
print(f'Logistic Regression Accuracy: {accuracy * 100:.2f}%')

KNN Accuracy: 84.68%
Naive Bais Accuracy: 71.09%
Logistic Regression Accuracy: 80.56%


6. Train the classifier

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Make predictions on the test data
predictions = knn.predict(X_test_bert_embeddings)
# Create a classification report
class_report = classification_report(test_labels, predictions, target_names=['negative', 'positive'])

# Create a confusion matrix
conf_matrix = confusion_matrix(test_labels, predictions)

# Print the classification report and confusion matrix
print("Classification Report:")
print(class_report)

print("\nConfusion Matrix:")
print(conf_matrix)

#predictions = rf.predict(test_embeddings)
predictions = NB.predict(X_test_bert_embeddings)
# Create a classification report
class_report = classification_report(test_labels, predictions, target_names=['negative', 'positive'])

# Create a confusion matrix
conf_matrix = confusion_matrix(test_labels, predictions)

# Print the classification report and confusion matrix
print("Classification Report:")
print(class_report)

print("\nConfusion Matrix:")
print(conf_matrix)

#predictions = rf.predict(test_embeddings)
predictions = lr.predict(X_test_bert_embeddings)
# Create a classification report
class_report = classification_report(test_labels, predictions, target_names=['negative', 'positive'])

# Create a confusion matrix
conf_matrix = confusion_matrix(test_labels, predictions)

# Print the classification report and confusion matrix
print("Classification Report:")
print(class_report)

print("\nConfusion Matrix:")
print(conf_matrix)




Classification Report:
              precision    recall  f1-score   support

    negative       0.88      0.80      0.84     36129
    positive       0.82      0.89      0.85     36483

    accuracy                           0.85     72612
   macro avg       0.85      0.85      0.85     72612
weighted avg       0.85      0.85      0.85     72612


Confusion Matrix:
[[28838  7291]
 [ 3836 32647]]
Classification Report:
              precision    recall  f1-score   support

    negative       0.77      0.60      0.67     36129
    positive       0.67      0.82      0.74     36483

    accuracy                           0.71     72612
   macro avg       0.72      0.71      0.71     72612
weighted avg       0.72      0.71      0.71     72612


Confusion Matrix:
[[21545 14584]
 [ 6407 30076]]
Classification Report:
              precision    recall  f1-score   support

    negative       0.82      0.78      0.80     36129
    positive       0.79      0.83      0.81     36483

    accuracy 

At present, knn is the best model on the full dataset. More tuning can be performed later to test whether logistic regression can perform better.